<a href="https://colab.research.google.com/github/nirb28/nn_catalyst/blob/main/src/different_approaches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os, sys

IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    print("Running in Colab!")
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    from google.colab import userdata
else:
    print("Not running in Colab.")

def resolve_path_gdrive(relativePath):
    if os.path.exists('/content/drive'):
        return '/content/drive/MyDrive/work/gdrive-workspaces/git/nn_catalyst/' + relativePath
    else:
        from utils import get_project_root
        return get_project_root() + "/" + relativePath

Running in Colab!
Mounted at /content/drive


In [4]:
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import AdamW

# Load the data
descriptors_path = 'descriptors.csv'
targets_path = 'compiled_data.csv'

In [5]:
descriptors_df = pd.read_csv(resolve_path_gdrive(descriptors_path))
targets_df = pd.read_csv(resolve_path_gdrive(targets_path))

<ipython-input-5-682bbed9e703>:1: DtypeWarning: Columns (90,91,92,93,94,95,96,97,98,138,139,140,147,148,149,165,166,167,174,175,176,183,184,185,189,190,191,192,193,194,201,202,203,210,211,212,219,220,221,228,229,230,234,235,236,237,238,239,240,241,242,297,298,299,300,301,302,303,304,305,342,343,344,345,346,347,354,355,356,363,364,365,381,382,383,390,391,392,399,400,401,405,406,407,408,409,410,417,418,419,426,427,428,435,436,437,444,445,446,450,451,452,453,454,460,461,462,468,469,470,484,485,486,492,493,494,500,501,502,506,507,508,509,510,516,517,518,524,525,526,532,533,534,540,541,542,546,547,548,549,550,556,557,558,564,565,566,580,581,582,588,589,590,596,597,598,602,603,604,605,606,612,613,614,620,621,622,628,629,630,636,637,638,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,833,870,871,875,883,886,887,894,902,1351,1452) have mixed types. Specify dtype option on

In [7]:
# Show sample rows
print("\nSample Rows from Descriptors DataFrame:")
print(descriptors_df.head())
print("\nSample Rows from Targets DataFrame:")
print(targets_df.head())


Sample Rows from Descriptors DataFrame:
   Label        ABC     ABCGG  nAcid  nBase             SpAbs_A  \
0   9268   4.719397  5.004088      0      0   6.720566232730447   
1  10488  10.334062  9.836417      0      0  16.752497538971177   
2  25579   5.875634  5.566041      0      0    9.43114762028933   
3   8952   6.611250  6.890735      1      0   10.68725972618713   
4  23681   7.249407  6.976306      0      0  11.945821561028193   

              SpMax_A           SpDiam_A              SpAD_A  \
0  2.1010029896154583  4.202005979230917   6.720566232730447   
1  2.3623398328574394  4.724679665714879  16.752497538971177   
2  2.1753277471610764  4.350655494322151    9.43114762028933   
3    2.28774942353935  4.425414875225794   10.68725972618713   
4  2.2671838628844996     4.534367725769  11.945821561028193   

              SpMAD_A  ...     SRW10     TSRW10          MW        AMW WPath  \
0  0.9600808903900638  ...  8.123558  33.343946  136.047505   6.802375    46   
1  1.196606

In [8]:
# selected column
selected_cols=[5, 14, 15, 23, 24, 25]
number_of_target_cols = len(selected_cols)
selected_cols.insert(0, 0)
targets_df = targets_df.iloc[:, selected_cols]
print(targets_df)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

       mol_num  dipole_n  max_spin_o  dipole_o  max_spin_r  dipole_r
0            1   4.63917    0.349349   7.63914    0.933582   8.80921
1            4   0.00005    0.280722   0.00000    0.472025   0.00001
2           11   3.31424    0.897425   2.75691    0.395451   1.28878
3           12   5.84638    0.838052   4.65300    0.258320   5.08854
4           13   4.96208    0.307085  10.87778    0.383838   6.90040
...        ...       ...         ...       ...         ...       ...
26228    34242   4.16353    0.189008   5.82675    0.216071   4.82770
26229    34243   5.55982    0.512668   9.34790    0.402216   6.53615
26230    34244   6.29952    0.460641   8.96955    0.241265   7.29014
26231    34245   3.52367    0.187718   8.74031    0.146653   4.98239
26232    34246   5.81051    0.249099   6.71696    0.254265   7.02366

[26233 rows x 6 columns]
cuda


In [9]:
# Keep only numeric columns
descriptors_numeric = descriptors_df.select_dtypes(include=['number'])
targets_numeric = targets_df.select_dtypes(include=['number'])

# Merge the numeric dataframes on the common label column
numeric_data = pd.merge(descriptors_numeric, targets_numeric, left_on='Label', right_on='mol_num')
numeric_data = numeric_data.drop(columns=['Label', 'mol_num'])

# Separate features and targets
X = numeric_data.iloc[:, :-number_of_target_cols]  # Assuming the last 30 columns are targets
y = numeric_data.iloc[:, -number_of_target_cols:]

In [10]:
# Apply variance threshold
selector = VarianceThreshold()
X_high_variance = selector.fit_transform(X)

# Convert to numpy arrays
X = X_high_variance
y = y.values

# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Standardize the data
scaler_X = StandardScaler().fit(X_train)
scaler_y = StandardScaler().fit(y_train)

X_train = scaler_X.transform(X_train)
X_val = scaler_X.transform(X_val)
X_test = scaler_X.transform(X_test)

y_train = scaler_y.transform(y_train)
y_val = scaler_y.transform(y_val)
y_test = scaler_y.transform(y_test)

# Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32, device=device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32, device=device)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32, device=device)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32, device=device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32, device=device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32, device=device)

In [11]:
# Create DataLoader for batch processing
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the individual model class
class SingleTargetNet(nn.Module):
    def __init__(self, input_size, dropout_rate=0.5):
        super(SingleTargetNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 1024)
        self.bn1 = nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512, 1)
        self.fc_skip = nn.Linear(1024, 512)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x1 = F.relu(self.bn1(self.fc1(x)))
        x1 = self.dropout(x1)

        x2 = F.relu(self.bn2(self.fc2(x1)))
        x2 = self.dropout(x2)

        # Skip connection
        x2 += self.fc_skip(x1)

        x3 = self.fc3(x2)
        return x3

def get_target5model():
    # Define the model
    model = nn.Sequential(
        nn.Linear(X_train.shape[1], 1024),
        nn.LeakyReLU(),
        nn.Linear(1024, 512),
        nn.LeakyReLU(),
        nn.Linear(512, 256),
        nn.LeakyReLU(),
        nn.Linear(256, 1)
    )
    return model

class RegressionNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(RegressionNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.relu = nn.LeakyReLU()
        self.dropout = nn.Dropout(0.1)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out = self.fc1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

In [14]:
# Function to train and evaluate individual models
def train_and_evaluate(target_index, model):
    model.to(device=device)
    criterion = nn.MSELoss()
    optimizer = AdamW(model.parameters(), lr=0.001)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)

    best_val_loss = np.inf
    patience_counter = 0
    num_epochs = 150

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs).squeeze()
            loss = criterion(outputs, targets[:, target_index])
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Target {target_index} - Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                outputs = model(inputs).squeeze()
                loss = criterion(outputs, targets[:, target_index])
                val_loss += loss.item()
        val_loss /= len(val_loader)
        print(f'Target {target_index} - Validation Loss: {val_loss}')

        scheduler.step(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), f'{selected_cols[target_index+1]}_{type(model).__name__}_model.pth')
        else:
            patience_counter += 1
            if patience_counter >= 15:
                print(f'Target {target_index} - Early stopping triggered')
                break

    model.load_state_dict(torch.load(f'{selected_cols[target_index+1]}_{type(model).__name__}_model.pth'))

    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs).squeeze()
            loss = criterion(outputs, targets[:, target_index])
            test_loss += loss.item()
    test_loss /= len(test_loader)
    print(f'Target {target_index} - Test Loss: {test_loss}')

    return model, test_loss

In [15]:
# Train and evaluate individual models for each target
test_losses = []
models = []
try_models = [SingleTargetNet(X_train.shape[1]), RegressionNetwork(X_train.shape[1], 512, 1), get_target5model()]

for target_index in range(y_train.shape[1]):
    for a_model in try_models:
        model, test_loss = train_and_evaluate(target_index, a_model)
        models.append(model)
        test_losses.append(test_loss)

Target 0 - Epoch 1/150, Loss: 0.8773315937326449
Target 0 - Validation Loss: 0.6787949778684755
Target 0 - Epoch 2/150, Loss: 0.7100362734096807
Target 0 - Validation Loss: 0.6704902547161754
Target 0 - Epoch 3/150, Loss: 0.7005765770175835
Target 0 - Validation Loss: 0.6312346545661368
Target 0 - Epoch 4/150, Loss: 0.6850633419050676
Target 0 - Validation Loss: 0.6376743200348645
Target 0 - Epoch 5/150, Loss: 0.6706144227759867
Target 0 - Validation Loss: 0.6332171061416951
Target 0 - Epoch 6/150, Loss: 0.6649495326891178
Target 0 - Validation Loss: 0.6199800150423516
Target 0 - Epoch 7/150, Loss: 0.6546979307356041
Target 0 - Validation Loss: 0.614047696314207
Target 0 - Epoch 8/150, Loss: 0.646941816911283
Target 0 - Validation Loss: 0.6057682277225866
Target 0 - Epoch 9/150, Loss: 0.6420653765430537
Target 0 - Validation Loss: 0.6057485040368104
Target 0 - Epoch 10/150, Loss: 0.6278365713703197
Target 0 - Validation Loss: 0.5856974735492613
Target 0 - Epoch 11/150, Loss: 0.62262476

<ipython-input-14-dff41d967af4>:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'{selected_cols[target_index+1]}_{type(model).__name__}_m

Target 0 - Test Loss: 0.5140507591933738
Target 0 - Epoch 1/150, Loss: 0.7373505125685436
Target 0 - Validation Loss: 0.6576094445658893
Target 0 - Epoch 2/150, Loss: 0.6613180566488243
Target 0 - Validation Loss: 0.5833908355817562
Target 0 - Epoch 3/150, Loss: 0.62095704088669
Target 0 - Validation Loss: 0.6015859987677598
Target 0 - Epoch 4/150, Loss: 0.6016235789511262
Target 0 - Validation Loss: 0.5681010717298927
Target 0 - Epoch 5/150, Loss: 0.5858812791227204
Target 0 - Validation Loss: 0.5675570652252291
Target 0 - Epoch 6/150, Loss: 0.5694697321415311
Target 0 - Validation Loss: 0.5852921797734935
Target 0 - Epoch 7/150, Loss: 0.5562712227153342
Target 0 - Validation Loss: 0.5401013086481792
Target 0 - Epoch 8/150, Loss: 0.5478530300826561
Target 0 - Validation Loss: 0.5491731199549466
Target 0 - Epoch 9/150, Loss: 0.5367443917046596
Target 0 - Validation Loss: 0.5662352704420323
Target 0 - Epoch 10/150, Loss: 0.5251315856216158
Target 0 - Validation Loss: 0.5604633950605625


KeyboardInterrupt: 

In [16]:
models

[SingleTargetNet(
   (fc1): Linear(in_features=780, out_features=1024, bias=True)
   (bn1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (fc2): Linear(in_features=1024, out_features=512, bias=True)
   (bn2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (fc3): Linear(in_features=512, out_features=1, bias=True)
   (fc_skip): Linear(in_features=1024, out_features=512, bias=True)
   (dropout): Dropout(p=0.5, inplace=False)
 ),
 RegressionNetwork(
   (fc1): Linear(in_features=780, out_features=512, bias=True)
   (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (relu): LeakyReLU(negative_slope=0.01)
   (dropout): Dropout(p=0.1, inplace=False)
   (fc2): Linear(in_features=512, out_features=512, bias=True)
   (bn2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (fc3): Linear(in_features=512, out_features=1, bias=True)
 ),
 Sequential(
  

In [17]:
# Prepare DataFrames for train, validation, and test predictions
train_df = pd.DataFrame()
val_df = pd.DataFrame()
test_df = pd.DataFrame()

r2_scores, rmse_scores, mae_scores = [], [], []

def compute_stats(target_index, df):
    print(df)
    observed_col = f'Observed_{target_index}'
    predicted_col = f'Predicted_{target_index}'

    # Calculate metrics
    observed = df[observed_col]
    predicted = df[predicted_col]
    r2 = r2_score(observed, predicted)
    rmse = mean_squared_error(observed, predicted, squared=False)
    mae = mean_absolute_error(observed, predicted)
    return r2, rmse, mae

In [19]:
def evaluate(target_index, model):
    # Make predictions on the train, validation, and test sets
    model.cpu().eval()
    with torch.no_grad():
        y_train_pred = model(X_train_tensor.cpu()).numpy()
        y_val_pred = model(X_val_tensor.cpu()).numpy()
        y_test_pred = model(X_test_tensor.cpu()).numpy()

    # Inverse transform the predictions and targets to their original scale
    y_train_pred_orig = scaler_y.inverse_transform(np.concatenate([np.zeros((y_train_pred.shape[0], target_index)), y_train_pred, np.zeros((y_train_pred.shape[0], y_train.shape[1] - target_index - 1))], axis=1))[:, target_index]
    y_val_pred_orig = scaler_y.inverse_transform(np.concatenate([np.zeros((y_val_pred.shape[0], target_index)), y_val_pred, np.zeros((y_val_pred.shape[0], y_val.shape[1] - target_index - 1))], axis=1))[:, target_index]
    y_test_pred_orig = scaler_y.inverse_transform(np.concatenate([np.zeros((y_test_pred.shape[0], target_index)), y_test_pred, np.zeros((y_test_pred.shape[0], y_test.shape[1] - target_index - 1))], axis=1))[:, target_index]

    y_train_orig = scaler_y.inverse_transform(y_train)[:, target_index]
    y_val_orig = scaler_y.inverse_transform(y_val)[:, target_index]
    y_test_orig = scaler_y.inverse_transform(y_test)[:, target_index]

    # Create dataframes for the predictions and actual values
    train_df[f'Observed_{target_index}'] = y_train_orig
    train_df[f'Predicted_{target_index}'] = y_train_pred_orig

    val_df[f'Observed_{target_index}'] = y_val_orig
    val_df[f'Predicted_{target_index}'] = y_val_pred_orig

    test_df[f'Observed_{target_index}'] = y_test_orig
    test_df[f'Predicted_{target_index}'] = y_test_pred_orig

    # Create and insert parity plots for train, validation, and test sets
    r2, rmse, mae = compute_stats(target_index, train_df)
    r2_scores.append(r2)
    rmse_scores.append(rmse)
    mae_scores.append(mae)

for target_index in range(y_train.shape[1]):
    for model_type in try_models:
        model = model_type
        model.load_state_dict(torch.load(f'{selected_cols[target_index+1]}_{type(model_type).__name__}_model.pth'))
        evaluate(target_index, model)


<ipython-input-19-84087f8d1cbc>:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'{selected_cols[target_index+1]}_{type(model_type).__name

       Observed_0  Predicted_0
0         3.03390     3.456505
1         2.21883     4.470612
2         3.42129     2.142875
3         6.54626     5.350793
4         2.70455     2.034520
...           ...          ...
20981     5.82050     5.254159
20982     2.90117     1.936822
20983     3.06542     3.045043
20984     2.26019     2.698958
20985     1.91199     2.564694

[20986 rows x 2 columns]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
<ipython-input-19-84087f8d1cbc>:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights

       Observed_0  Predicted_0
0         3.03390     3.538626
1         2.21883     4.486116
2         3.42129     1.889467
3         6.54626     4.863782
4         2.70455     2.309811
...           ...          ...
20981     5.82050     5.852612
20982     2.90117     2.274118
20983     3.06542     2.909958
20984     2.26019     2.358000
20985     1.91199     2.619898

[20986 rows x 2 columns]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
<ipython-input-19-84087f8d1cbc>:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights

...

       Observed_0  Predicted_0  Observed_1  Predicted_1
0         3.03390     3.916124    0.498762     0.342581
1         2.21883     4.377955    0.162431     0.201065
2         3.42129     2.498786    0.547589     0.486283
3         6.54626     5.610939    0.308542     0.289868
4         2.70455     1.833646    0.572807     0.567159
...           ...          ...         ...          ...
20981     5.82050     5.346223    0.342817     0.375852
20982     2.90117     1.533721    0.485546     0.447304
20983     3.06542     3.034153    0.210382     0.191891
20984     2.26019     2.620820    0.336572     0.322709
20985     1.91199     2.391123    0.211747     0.235239

[20986 rows x 4 columns]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
<ipython-input-19-84087f8d1cbc>:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights

       Observed_0  Predicted_0  Observed_1  Predicted_1  Observed_2  \
0         3.03390     3.916124    0.498762     0.342581     3.89562   
1         2.21883     4.377955    0.162431     0.201065     7.08420   
2         3.42129     2.498786    0.547589     0.486283     6.47881   
3         6.54626     5.610939    0.308542     0.289868     5.39737   
4         2.70455     1.833646    0.572807     0.567159    11.06172   
...           ...          ...         ...          ...         ...   
20981     5.82050     5.346223    0.342817     0.375852     6.03192   
20982     2.90117     1.533721    0.485546     0.447304     3.37806   
20983     3.06542     3.034153    0.210382     0.191891    11.38933   
20984     2.26019     2.620820    0.336572     0.322709     1.31823   
20985     1.91199     2.391123    0.211747     0.235239     2.01098   

       Predicted_2  
0         3.154392  
1         5.779972  
2         5.212570  
3         9.927725  
4        10.053726  
...            ...  


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
<ipython-input-19-84087f8d1cbc>:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights

FileNotFoundError: [Errno 2] No such file or directory: '15_RegressionNetwork_model.pth'

In [20]:
r2_scores, rmse_scores, mae_scores

([0.602973221905207,
  0.5993249085928642,
  0.6117650030599211,
  0.8808057793434411,
  0.9528224754377352,
  0.8022839179780921,
  0.5932465964354097],
 [1.2525446674336702,
  1.2582863778270408,
  1.2385988244759507,
  0.04086779424231646,
  0.02571113055942406,
  0.052634996317358684,
  1.9412134364527274],
 [0.9528549375792478,
  0.9561028144634287,
  0.9415593225120522,
  0.03026810860835579,
  0.01827938131057719,
  0.03904015856283628,
  1.4662363570828618])